In [1]:
#importing packages and reading statements
import pandas as pd
income_statement = pd.read_json("https://fmpcloud.io/api/v3/income-statement/AAPL?limit=120&apikey=demo")
income_statement.set_index('date', inplace=True)
cash_flow = pd.read_json("https://fmpcloud.io/api/v3/cash-flow-statement/AAPL?limit=120&apikey=demo")
cash_flow.set_index('date', inplace=True)

We only need revenue, Earnings per share and free cash flow

In [2]:
data = income_statement[['revenue', 'eps']].copy()
data['fcf'] = cash_flow['freeCashFlow']

In [3]:
data

,revenue,eps,fcf
date,,,
2022-09-24,394328000000,6.150000,1.114430e+11
2021-09-25,365817000000,5.670000,9.295300e+10
2020-09-26,274515000000,3.310000,7.336500e+10
2019-09-28,260174000000,2.992500,5.889600e+10
2018-09-29,265595000000,3.002500,6.412100e+10
2017-09-30,229234000000,2.317500,5.080300e+10
2016-09-24,215639000000,2.087500,5.227600e+10
2015-09-26,233715000000,2.320000,6.977800e+10
2014-09-27,182795000000,1.622500,4.990000e+10


Calculating percentage change

In [4]:
data = data.sort_index()
data['rev %-chg'] = data['revenue'].pct_change()
data['eps %-chg'] = data['eps'].pct_change()
data['fcf %-chg'] = data['fcf'].pct_change()

In [7]:
data['rev %-chg'].head()

date
1985-09-30         NaN
1986-09-30   -0.008549
1987-09-30    0.399180
1988-09-30    0.529969
1989-09-30    0.297834
Name: rev %-chg, dtype: float64

In [8]:
data['rev %-chg'].tail()

date
2018-09-29    0.158620
2019-09-28   -0.020411
2020-09-26    0.055121
2021-09-25    0.332594
2022-09-24    0.077938
Name: rev %-chg, dtype: float64

In [9]:
data['eps %-chg'].head()

date
1985-09-30         NaN
1986-09-30    1.400090
1987-09-30    0.383330
1988-09-30    0.855475
1989-09-30    0.149345
Name: eps %-chg, dtype: float64

In [10]:
data['eps %-chg'].tail()

date
2018-09-29    0.295577
2019-09-28   -0.003331
2020-09-26    0.106099
2021-09-25    0.712991
2022-09-24    0.084656
Name: eps %-chg, dtype: float64

Only need 10 years of data for analysis

In [11]:
#setting the index uptill 10 years from 2022
data_set = data.loc['2010-01-01':]
data_set.index = data_set.index.year
rev = data_set[['revenue', 'rev %-chg']].copy()
eps = data_set[['eps', 'eps %-chg']].copy()
fcf = data_set[['fcf', 'fcf %-chg']].copy()

In [12]:
#creating an excel file
def create_sheet(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
writer = pd.ExcelWriter("financials.xlsx", engine='xlsxwriter')
workbook = writer.book
create_sheet(writer, rev, 'Revenue')
create_sheet(writer, eps, 'EPS')
create_sheet(writer, fcf, 'FCF')
workbook.close()

In [17]:
#adding charts in excel file
def create_sheet(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
    worksheet = writer.sheets[sheet_name]
    workbook = writer.book
      
    chart = workbook.add_chart({'type': 'line'})
    
    chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 1, 11, 1],
    })
    
    column_chart = workbook.add_chart({'type': 'column'})
    
    column_chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 2, 11, 2],
        'y2_axis': True,
    })
    
    chart.combine(column_chart)
    chart.set_title({'name': sheet_name})
    chart.set_x_axis({'name': 'Date'})
    chart.set_y_axis({'name': 'Value'})
    column_chart.set_y2_axis({'name': '%-chg'})
    
    worksheet.insert_chart("E1", chart)